In [18]:
import numpy as np

In [19]:
# Loading all the datasets

X_seen=np.load('X_seen.npy',encoding='bytes',allow_pickle=True) #	(40 x N_i x D): Seen training class

Attr=np.load('class_attributes_seen.npy',encoding='bytes',allow_pickle=True) # (40 x 85): Seen class attributes

UAttr=np.load('class_attributes_unseen.npy',encoding='bytes',allow_pickle=True) # (10 x 85): Unsee class atrributes

Xtest=np.load('Xtest.npy',encoding='bytes',allow_pickle=True)
Ytest=np.load('Ytest.npy',encoding='bytes',allow_pickle=True)

In [20]:
# Calculating mean of the each of these classes

Mean = np.zeros((40,4096)) #Mean will store all the 40 means


for i in range(40):
    Mean[i] = np.mean(X_seen[i],axis=0)

print("Shape of Mean is ",Mean.shape)
# printing shape for cross validation

Shape of Mean is  (40, 4096)


Calculating the Matrix $M$ of shape $10 \times 40$ which contains $S_{(c,k)}$'s  
of all the 10 classes for all 40 seen classes  
$$M_{attr} = A_{unseen}A_{seen}^T$$

In [21]:
MAtrr = np.matmul(UAttr,Attr.T) 
#MAttr is of shape (10,40) each row containg all the Sc,k without normalizing
print("Shape of MAttr is ",MAtrr.shape)

Shape of MAttr is  (10, 40)


In [22]:
# Normalizing the Matrix MAttr

MAtrr = MAtrr.astype(float) #converting values to floating point before normalization
row_Sum = np.sum(MAtrr,axis=1,dtype=float) #calculation the \sum(Sc,l) for each of the 40 l'set

# Normalizing the values

for i in range(row_Sum.shape[0]):
    for j in range(MAtrr.shape[1]):
        MAtrr[i][j] = MAtrr[i][j]/row_Sum[i]

# cross validating sum of first row
sum_row1 = np.sum(MAtrr[0],dtype=float)
print("sum of all the values of row 1 of M is ",sum_row1)

# ideally the sum should be 1

sum of all the values of row 1 of M is  1.0


### **Calculating the mean of unseen classes**  
for each unseen class $c$ we have,

$$ \mu_c = \sum_{k=1}^{40} s_{c,k}\mu_k$$

Let's calculate the matrix $M_{\mu}$ of shape $10 \times 4096$  
containing mean $\mu_c$'s for all the unseen classes

$$ \implies M_{\mu} = M_{attr}X_{\mu} $$ 

<center>

where, $M_{\mu} = $ 
                        $ \begin{bmatrix}
                            \mu_{41} \\
                            \mu_{42} \\ 
                            .      \\  
                            .      \\
                            \mu_{50} \\
                        \end{bmatrix}
                        $

Also, $X_{\mu} = $
                        $ \begin{bmatrix}
                            \mu_{1} \\
                            \mu_{2} \\ 
                            .      \\  
                            .      \\
                            \mu_{40} \\
                        \end{bmatrix}
                        $

</center>

and $M_{attr}$ is the matrix containing normalized $S_{c,k}$'s calculated above


In [23]:
# UMean will store the mean of unseen classes
# UMean = MAttr x Mean, UMean will be of shape 10,40

UMean = np.matmul(MAtrr,Mean)
print("shape of UMean is ",UMean.shape)

shape of UMean is  (10, 4096)


### **Testing time**

for each test input $x_*$ calculate $\lVert x_*-\mu_{c} \rVert^2$ for each unseen class $c$

In [24]:
N = Xtest.shape[0] # number of testing inputs

D = np.ones((N,10),dtype=float) #this will have the all the corresponsing norms

for i in range(N):
    xi = Xtest[i]
    for uc in range(UMean.shape[0]):
        D[i][uc] = np.linalg.norm(xi-UMean[uc])

print("shape of D is",D.shape)

shape of D is (6180, 10)


In [25]:
# getting the predictions
Ypredicted = np.argmin(D,axis = 1) + 1 #adding to make it 1 based indexing

print("shape of Ypredicted is ",Ypredicted.shape)

shape of Ypredicted is  (6180,)


In [26]:
# reshaping the array of true values
Ytrue = Ytest.reshape(-1,)

# calculating the correct prediction
equals = (Ypredicted == Ytrue)
Corrects = np.sum(equals)

print(f"Accuracy of the model is {Corrects*100/N} %")

Accuracy of the model is 46.89320388349515 %
